# 개요
## 추진배경
* 긴급하게 현재 가지고 있는 데이터로, 예측관련하여 만들 수 있는 머신러닝 모델을 요청받음
* Kaggle에서 운송수단을 예측하는 KNN모델 샘플을 발견하여 적용

## 계획
* 국내외의 이동 데이터를 학습한 머신러닝을 활용하여, 조건(국가, 거래업체 등)입력시 어떤 운송수단으로 수출할지 예측(항공/해상)
  * [모델1] KNeighborsClassifier 활용한 분류
    * 모델 정확성을 위해 7개의 Feature로 학습
    * 업무 사용시에는 사용자가 알 수 있는 4가지 정보를 필수로 받음 ('Loading Country', 'Final Destination country', 'Sold-to party', 'Ship To Party')
      * 3가지 추가정보 제공시 더 정확한 예측제공, 없는 경우 기존데이터의 최빈값으로 대체하여 예측 (Sales Organization, Incoterms, Dangerous goods)

* 예측된 운송수단을 기반으로, 어느정도의 공간(부피)가 필요한지 확인 후 부킹(운송을 위한 공간예약)에 활용
  * [모델2] 활용한 예측
    * 모델1에서 입력/예측된 값을 기반으로 부피(Volumn)을 예측
      * 'Loading Country', 'Final Destination country', 'Sold-to party', 'Ship To Party' (+ 'AIR/VESSEL', 'Dangerous goods')

* 예측된 값과 함께, 기존 데이터의 History를 제공해 어떤 출발지에서 나갔었는지 함께 공유하여 업무에 활용
  * 어떤 출발지인지에 따라, 사용하면 안되는 선사(해상운송업체)나 항공사가 정해질 수 있음

* KNN 참조한 샘플코드 : https://www.kaggle.com/code/sergeifursa/shipment-type-prediction-with-knn

## 효과
* 특정 조건(출/도착국가, 상대업체)을 기반으로, 항공기와 배 중 어떤 수단으로 얼마만큼 물량을 선적해야할 지 예측
* 예측한 데이터를 기반으로, 조기에 Space(선적공간)을 수배하고 선점하여 대응력 강화

## github repository
* 별도의 github레포로 정리하지 않고, 하단에 코드로만 기록

## [세부내용] 구현내용 & 사용한 언어/패키지 등
(1) db로 전환하여 `sqlite3`으로 데이터 보관
(2) 데이터 전처리는 아래의 로직에 따라 `pandas`로 처리
  * Drop
    * 예측 대상인 중량(GrossWT), 항공중량(ChargeableWT), 부피(Volumn)이 모두 없으면 Drop
    * 배로 선적하는데 부피가 없는 경우 Drop
  * 변환(단위 통일)
    * 중량은 G/KG 중 KG으로, 부피는 CCM/CBM중 CBM으로 설정
  * 대체(Null보완)
    * 항공중량(ChargeableWT)만 없는 경우, 별도의 계산식으로 보완처리 (Max(중량, 부피*167))
  * 수치변환(To Numeric)
    * Y값으로 사용할 값 중 Categorical한 AIR/VESSEL컬럼은 dict로 관리 및 보관
    * X값으로 사용할 값 중 Categorical한 컬럼들은 Scikit-learn의 `category_encoders`로 변환
      * 향후 데이터가 추가/삭제될 상황을 위해 사용(업체코드 등은 추가가 빈번할 것으로 예상)
      * High Cardinality하여 OnehotEncoding으로 인한 차원문제도 방지
      * 최초 실행 후 `joblib`로 encoder값을 저장해두고, 이후에는 로딩하여 사용
(3) 데이터셋 분할은 `Scikit-learn`의 `train_test_split`으로 train/test 8:2로 split
(4) 모델 파라미터는 optuna를 활용하여 세팅
  * 첫 사용 모델로 일부 특이사항만 지정하여 `OptunaSearchCV`로 파라미터 지정
    * 서열형(Ordinal)이 아닌 Categorical한 Y값이어서 Hamming을 metric으로 지정
    * 일부 Null값이 있어 kd_tree에서는 적용불가 메시지가 떠 kd_tree는 대상에서 제외
    * 이외 파라미터는 공식문서의 default값을 기준으로 일부 buffer를 두어 세팅함
(5) `Scikit-learn`의 `cross_val_score`으로 모델평가 (accuracy사용)
(6) 모델예측은 앞서 `category_encoders`으로 변환 후 학습했으므로, predict시에도 변환 후 적용
(7) joblib로 모델저장
(8) 이후 내용 추가 예정

# 데이터로딩

## Load with DB(sqlite3)

In [ ]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('sample.db')

# Write the DataFrame to the database
query = "SELECT * FROM `table`"
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()
df

# 데이터 전처리(Date Preprocessing)

## Drop 및 값 변환
* [Drop] 운임 또는 Space확보의 기준되는 값이 없는 것이므로 Drop
  * 해상/항공여부를 예측하더라도, Space정보가 없는 경우는 이후 대응이 불가하므로 Drop
* [변환] 중량 등 수치에 대한 단위 일치
  * 중량(KG, G), 부피(CBM, CCM)일치시키기
* [Null대체] 주어진 값으로 계산하여 대체 가능한 경우 반영
  * Chargeable WT : 중량과 부피를 기준으로 계산가능. Null인 경우 계산하여 대체 

In [ ]:
# 판다스 컬럼생략하지 않게하는 옵션설정
pd.set_option('display.max_columns', None)

In [ ]:
def drop_fillna_convert_df(df):
    # Drop
    df_dropped = df.drop(df[(df['AIR/VESSEL']==2) & df['Charg. Unit'].isna()].index).copy()
    ## 모든 중량/부피가 없음
    df_dropped = df_dropped.drop(df_dropped[df_dropped['Gross weight'].isna() & df_dropped['Volume'].isna() & df_dropped['Charg. weigh'].isna()].index)
    ## 해상인데 부피 또는 부피단위가 없음
    df_dropped = df_dropped.drop(df_dropped[(df_dropped['AIR/VESSEL']==1) & df_dropped['Volume'].isna()].index)
    df_dropped = df_dropped.drop(df_dropped[(df_dropped['AIR/VESSEL']==1) & df_dropped['Volume unit'].isna()].index)
    ## 항공인데, ChargeableWT가 없는데, 계산할 기초값인 Groww weight나 Volumn 중 하나가 없음
    df_dropped = df_dropped.drop(df_dropped[(df_dropped['AIR/VESSEL']==2) & df_dropped['Charg. weigh'].isna() & df_dropped['Gross weight'].isna()].index)
    df_dropped = df_dropped.drop(df_dropped[(df_dropped['AIR/VESSEL']==2) & df_dropped['Charg. weigh'].isna() & df_dropped['Volume'].isna()].index)

    # Convert Unit
    df_dropped.loc[df_dropped['Gross unit'] == 'G', 'Gross weight'] *= 1000
    df_dropped.loc[df_dropped['Volume unit'] == 'CCM', 'Volume'] /= 1000

    # Null대체
    crit_replace_na = (df_dropped['Charg. weigh'].isnull()) & (df_dropped['AIR/VESSEL'] == 2)
    df_dropped.loc[crit_replace_na, 'Charg. weigh'] = np.maximum(df_dropped[crit_replace_na]['Gross weight'], df_dropped[crit_replace_na]['Volume'] * 167)

    return df_dropped

df_dropped = drop_fillna_convert_df(df)

## X, Y값 변환(To numeric)
* Y값 변환
  * 사용할 3개의 Y값(AIR/VESSEL, Volumn, Charg. weigh)중, 한가지는 Categorical이므로 숫자로 변경
    * AIR/VESSEL(Categorical이므로), Volumn(Numeric), Charg. weigh(Numeric)
  * 해상/항공만 사용할 예정이므로 해상과 항공에 대해 매핑, 나머지는 9

* X값 변환
  * Dangerous goods는 2가지 값(O,X)밖에 없으므로 0,1로 변환

In [ ]:
mapping_airvessel_dict = dict()

for i in df['AIR/VESSEL'].unique():
    if 'S' in i:
        mapping_airvessel_dict[i] = 1
    elif 'P' in i or 'M' in i:
        mapping_airvessel_dict[i] = 2
    else:
        mapping_airvessel_dict[i] = 9
mapping_airvessel_dict

In [ ]:
# y값 매핑
df_dropped['AIR/VESSEL'] = df_dropped['AIR/VESSEL'].map(mapping_airvessel_dict)
# X값 매핑
df_dropped['Dangerous goods'] = df_dropped['Dangerous goods'].map({None:0,'X':1})

## X값 변환(To numeric with LeaveOneOut Encoder)
* 머신러닝학습을 위해 숫자로 변환 필요
* LeaveOneOut Encoder를 활용하는 것으로 결정
  * 국가코드는 거의 바뀔 일이 없지만, 아프리카 지역 등 일부 Minor한 국가로 판매할 경우 추가될 수 있음
  * 업체코드는 추가/삭제될 가능성이 높음
  * Unique값이 많으므로 One-hot Encoding시 차원이 너무 많아질 위험 있음(High Cardinality)
  * 처음에는 아래와 같이 dict로 각 컬럼별 매핑값을 관리하고자했으나, 번거로움과 대상이 많아짐에 따라 Encoder를 활용
    ```python
      party_code_reverse = {v: k for k, v in party_code.items()}
      country_code_map_reverse = {v: k for k, v in country_code_map.items()}

      x_data_encoded = pd.DataFrame()
      for each_column in x_column:
          if each_column in ['Loading Country', 'Final Destination country']:
              x_data_encoded[each_column] = x_data[each_column].map(country_code_map_reverse)
          elif each_column in ['Sold-to party', 'Ship To Party']:
              x_data_encoded[each_column] = x_data[each_column].map(party_code_reverse)
    ```
* Sci-kit learn LeaveOneOut Encoder 공식문서
  * https://contrib.scikit-learn.org/category_encoders/leaveoneout.html

In [ ]:
# 학습할 Feature
x_column = ['Loading Country', 'Final Destination country', 'Sold-to party', 'Ship To Party', 'Dangerous goods','Sales Organization','Incoterms']
y_column = 'AIR/VESSEL'

In [ ]:
# LeaveOneOutEncoder (최초 실행시)
import category_encoders as ce
import joblib

encoder_leave_one_out = ce.LeaveOneOutEncoder(cols=x_column, sigma=0.1, return_df=True)
x_data_all = encoder_leave_one_out.fit_transform(df_dropped[x_column], df_dropped[y_column])

joblib.dump(encoder_leave_one_out, 'encoder_leave_one_out.pkl')

x_data_all

,Loading Country,Final Destination country,Sold-to party,Ship To Party,Dangerous goods,Sales Organization,Incoterms
0,7.047596,3.679223,3.861376,8.134116,2.765859,2.988212,3.206009
1,1.501914,1.956254,1.396398,1.010231,2.410832,3.551036,2.268934
2,1.246987,1.247269,1.076889,1.069540,2.266524,3.929307,1.368437
3,7.557798,3.385827,4.343922,5.881165,2.453504,3.253987,3.082357
4,1.711316,1.644219,1.418986,1.081862,2.604062,1.513666,1.461735
...,...,...,...,...,...,...,...
68896,1.805997,1.211290,1.620240,0.923400,2.128367,4.805674,1.252805
68897,1.738878,1.543501,1.904886,0.850793,2.625204,2.538548,2.008110
68898,1.916489,1.649348,1.104931,1.064014,2.018030,2.863359,1.569362
68899,1.903145,2.041549,2.386892,0.954536,2.133451,1.841485,1.714075


In [ ]:
# LeaveOneOutEncoder (실행내역 있는 경우)

encoder_leave_one_out = joblib.load('encoder_leave_one_out.pkl')
x_data_all = encoder_leave_one_out.transform(df_dropped[x_column], df_dropped[y_column])

joblib.dump(encoder_leave_one_out, 'encoder_leave_one_out.pkl')

x_data_all

,Loading Country,Final Destination country,Sold-to party,Ship To Party,Dangerous goods,Sales Organization,Incoterms
0,7.041479,2.808034,3.644974,10.297530,2.271593,3.376236,2.761253
1,1.203195,1.595750,1.537957,1.012471,2.542451,4.059490,1.791253
2,1.509811,1.355136,1.029430,0.941528,2.484937,3.039443,1.278045
3,8.488335,4.329470,5.074977,5.995380,2.378870,3.362426,2.950839
4,1.371701,1.669704,1.662936,1.251279,2.103728,1.285509,1.482815
...,...,...,...,...,...,...,...
68896,1.759238,1.000788,1.684182,1.115493,2.130670,2.516951,1.334155
68897,2.000342,1.765736,1.469979,1.130242,2.640289,2.358999,1.532509
68898,1.901704,1.555484,1.386904,0.944819,2.572425,2.429427,1.479614
68899,2.164074,2.300433,2.976153,1.097528,2.626010,1.895660,1.757920


In [ ]:
y_data_all = df_dropped['AIR/VESSEL']
y_data_all

0        9
1        1
2        1
3        9
4        1
        ..
68896    1
68897    1
68898    1
68899    1
68900    1
Name: AIR/VESSEL, Length: 68880, dtype: int64

## 데이터셋 분할(Split Train/Test)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data_all, y_data_all, train_size=0.8, shuffle=True, stratify=y_data_all)

# 모델 학습 (KNeighborsClassifier)

## KNeighborsClassifier 학습
* 처음 사용해보는 모델이면서 마감기한이 촉박하여, optuna를 활용하여 파라미터 세팅
* Scikit-learn KNeighborsClassifier 공식문서
  * https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
* Metric의 경우, hamming distance를 채택
  * Kaggle 샘플코드에서 Categorical인 경우는 hamming distance를 기준한다고 되어있음을 참고
  * 시간관계상 간단히 알아보니, Simple matching이라면 hamming, 중요도가 있다면 jacard라고 함
  * 이외에 변수가 서열형(ordinal)인 경우에도 다른 방법이 있다고 하나, 현재의 데이터는 서열형으로 볼 수는 없어 제외

In [ ]:
import optuna
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(metric='hamming')

param_distributions = {
    "n_neighbors": optuna.distributions.IntDistribution(3, 5),
    "weights": optuna.distributions.CategoricalDistribution(['uniform', 'distance']),
    "algorithm": optuna.distributions.CategoricalDistribution(['auto', 'ball_tree', 'brute']), #'kd_tree'는 nan인 경우 문제가 있어 제외
    "leaf_size" : optuna.distributions.IntDistribution(20, 40),
}

optuna_search = optuna.integration.OptunaSearchCV(
    clf, 
    param_distributions, 
    n_jobs=-1, # Number of parallel jobs. -1 means using all processors.
    cv=5, #  estimator가 classifier & label이 binary or multiclass라면 sklearn.model_selection.StratifiedKFold 적용 (이외는 sklearn.model_selection.KFold)
    n_trials=100, 
    timeout=600, 
    verbose=2,
    scoring=None, # If None, score on the estimator is used.
    refit=True # Best Parameter로 refit. refitted estimator는 best_estimator_ attribute로 바로 predict가능
)

optuna_search.fit(x_train, y_train)

print("Best trial:")
trial = optuna_search.study_.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

py:13: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2024-08-04 22:23:30,602] A new study created in memory with name: no-name-8095b4dc-5e03-4ae4-942f-6fa9c1f72a9c
[I 2024-08-04 22:24:37,815] Trial 4 finished with value: 0.5235917538057604 and parameters: {'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 24}. Best is trial 4 with value: 0.5235917538057604.
[I 2024-08-04 22:24:39,034] Trial 5 finished with value: 0.5235917538057604 and parameters: {'n_neighbors': 5, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 36}. Best is trial 4 with value: 0.5235917538057604.
[I 2024-08-04 22:24:51,518] Trial 6 finished with value: 0.5235917538057604 and parameters: {'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 23}. Best is trial 4 with value: 0.5235917538057604.
[I 2024-08-04 22:26:20,120] Trial 3 finish

Best trial:
  Value:  0.5235917538057604
  Params: 
    n_neighbors: 5
    weights: distance
    algorithm: brute
    leaf_size: 24


In [ ]:
optuna_search.best_params_

{'n_neighbors': 5,
 'weights': 'distance',
 'algorithm': 'brute',
 'leaf_size': 24}

In [ ]:
best_model_knclassifier = optuna_search.best_estimator_
best_model_knclassifier

KNeighborsClassifier(algorithm='brute', leaf_size=24, metric='hamming',
                     weights='distance')

## KNeighborsClassifier 모델평가

In [ ]:
import sklearn.model_selection

sklearn.model_selection.cross_val_score(best_model_knclassifier, x_test, y_test, scoring='accuracy', cv=5, 
                                        n_jobs=None, verbose=0)

array([0.52358491, 0.52377495, 0.52377495, 0.52341198, 0.52341198])

## KNeighborsClassifier 예측(Prediction)

In [ ]:
# 예측대상 입력 후 변환
## 입력
x_test_input = pd.DataFrame({'Loading Country':'MX',
                             'Final Destination country':'US',
                             'Sold-to party':'C310-00',
                             'Ship To Party':'4891788',
                             'Dangerous goods':'0',
                             'Sales Organization':'R001',
                             'Incoterms':'CIP'},
                             index=[0]
)
## LeaveOneOutEncoder변환
encoder_leave_one_out = joblib.load('encoder_leave_one_out.pkl')
x_data_to_predict = encoder_leave_one_out.transform(x_test_input[x_column])
x_data_to_predict['Dangerous goods'] = x_data_to_predict['Dangerous goods'].map({np.nan:0})

x_data_to_predict

,Loading Country,Final Destination country,Sold-to party,Ship To Party,Dangerous goods,Sales Organization,Incoterms
0,7.339717,3.507706,4.077912,9.0,0,3.443488,3.025576


In [ ]:
# 예측값 {1:'VESSEL',2:'AIR'}

prediction = best_model_knclassifier.predict(x_data_to_predict)
prediction

array([2], dtype=int64)

In [ ]:
array_to_conver = prediction
np.where(array_to_conver == 1, 'VESSEL', np.where(array_to_conver == 2, 'AIR', 'UNKNOWN'))

array(['AIR'], dtype='<U7')

## KNeighborsClassifier 모델저장

In [ ]:
import joblib

joblib_file = "model_knn_1st_fitted.joblib"
joblib.dump(best_model_knclassifier, joblib_file)

['model_knn_1st_fitted.joblib']

In [ ]:
joblib_file = "model_knn_1st_fitted.joblib"
knn_loaded = joblib.load(joblib_file)

prediction = knn_loaded.predict(x_data_to_predict)
prediction

array([2], dtype=int64)

In [ ]:
array_to_conver = prediction
np.where(array_to_conver == 1, 'VESSEL', np.where(array_to_conver == 2, 'AIR', 'UNKNOWN'))

array(['AIR'], dtype='<U7')